In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision import datasets

import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [16]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    train_loss,train_acc=0,0
    for batch,(X,y) in enumerate(dataloader):
        y_pred=model(X)
        loss=loss_fn(y_pred,y)
        train_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    
    train_loss=train_loss/len(dataloader)
    train_acc=train_acc/len(dataloader)
    return train_loss,train_acc

In [4]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
    model.eval()
    test_loss,test_acc=0,0
    with torch.inference_mode():
        for batch,(X,y) in enumerate(dataloader):
            test_pred_logits=model(X)
            loss=loss_fn(test_pred_logits,y)
            test_loss+=loss.item()
            test_pred_labels=test_pred_logits.argmax(dim=1)
            test_acc+=(((test_pred_labels==y).sum().item())/len(test_pred_labels))
    test_loss=test_loss/len(dataloader)
    test_acc=test_acc/len(dataloader)
    return test_loss,test_acc

In [5]:
path='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato'

In [6]:
train_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

In [7]:
train_data=datasets.ImageFolder(root=os.path.join(path,'train'),transform=train_transform)
test_data=datasets.ImageFolder(root=os.path.join(path,'test'),transform=test_transform)

In [8]:
len(train_data),len(test_data)

(294, 97)

In [9]:
class_names=train_data.classes
class_dict=train_data.class_to_idx

class_names,class_dict

(['apples', 'tomatoes'], {'apples': 0, 'tomatoes': 1})

In [10]:
from tqdm import tqdm
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int):
    
    results={'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
    
    for i in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        test_loss,test_acc=test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn)
        print(f'Epoch {i} | Train loss: {train_loss:.4f},Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f},Test acc: {test_acc:.4f}')
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results

In [11]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(3,6,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(6,16,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),
            nn.Linear(54*54*16,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,len(class_names))
        )

    def forward(self,X):
        return F.log_softmax(self.conv(X))

In [12]:
torch.manual_seed(101)
train_loader=DataLoader(dataset=train_data,batch_size=32,shuffle=True,num_workers=os.cpu_count())
test_loader=DataLoader(dataset=test_data,batch_size=32,shuffle=False,num_workers=os.cpu_count())

In [35]:
torch.manual_seed(101)
model=Model()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [36]:
torch.manual_seed(42)

n=20
model_results=train(
    model=model,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

  5%|▌         | 1/20 [00:02<00:47,  2.50s/it]

Epoch 0 | Train loss: 0.6538,Train acc: 0.6427 | Test loss: 0.6184,Test acc: 0.7266


 10%|█         | 2/20 [00:04<00:42,  2.38s/it]

Epoch 1 | Train loss: 0.6448,Train acc: 0.6594 | Test loss: 0.5696,Test acc: 0.7734


 15%|█▌        | 3/20 [00:07<00:40,  2.36s/it]

...

Epoch 17 | Train loss: 0.3302,Train acc: 0.8604 | Test loss: 0.3826,Test acc: 0.8438


 95%|█████████▌| 19/20 [00:45<00:02,  2.42s/it]

Epoch 18 | Train loss: 0.2967,Train acc: 0.8844 | Test loss: 0.4296,Test acc: 0.7734


100%|██████████| 20/20 [00:47<00:00,  2.39s/it]

Epoch 19 | Train loss: 0.3002,Train acc: 0.8615 | Test loss: 0.3742,Test acc: 0.8516


In [37]:
index=list(range(97))

c=0
for i in index:
    model.eval()
    with torch.no_grad():
        pred=model(test_data[i][0].view(1,3,224,224)).argmax()

    print(f'Actual: {class_names[test_data[i][1]]} | Predicted: {class_names[pred.item()]}')
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: apples
Actual: apples | Predicted: tomatoes
Actual: 

In [51]:
def tester(model: nn.Module,
           image: str,
           transform: torchvision.transforms.transforms,
           class_names: list):
    
    image = torchvision.io.read_image(image)
    
    # Check if the image has only one channel (grayscale)
    if image.shape[0] == 1:
        # Convert grayscale image to RGB by duplicating the single channel
        image = torch.cat([image, image, image], dim=0)
    
    image = image.type(torch.float32) / 255
    image_new = transform(image)
    
    model.eval()
    with torch.inference_mode():
        logit = model(image_new.unsqueeze(0))
    
    prob = torch.softmax(logit, dim=1)
    label = class_names[torch.argmax(prob, dim=1)]
    print(f'Predicted breed: {label}')

In [52]:
apple1='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/apple1.jpg'
apple2='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/apple2.jpg'
apple3='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/apple3.jpg'

tomato1='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/tomato1.jpg'
tomato2='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/tomato2.jpg'
tomato3='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Apple tomato/tomato3.jpg'

In [53]:
transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
   
])

In [54]:
test_class=[apple1,apple2,apple3,tomato1,tomato2,tomato3]

for i in test_class:
    
    tester(model=model,
           image=i,
           transform=transform,
           class_names=class_names)

Predicted breed: apples
Predicted breed: apples
Predicted breed: apples
Predicted breed: tomatoes
Predicted breed: tomatoes
Predicted breed: tomatoes
